In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [31]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [8]:
sac_rt_itp_id = 273

In [9]:
analysis_date = dt.date(2022, 2, 8)

In [10]:
pbar = tqdm()

0it [00:00, ?it/s]

In [11]:
sac_rt = rt.OperatorDayAnalysis(sac_rt_itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found_parquet
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
could not generate delays for trip 991293
Columns must be same length as key
could not generate delays for trip 993942
Columns must be same length as key


In [14]:
sac_rt.set_filter(start_time='15:00', end_time='19:00')

In [32]:
m = rt.OperatorDayAnalysis.segment_speed_map(sac_rt)

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}
speed above 80 for shape 41007, dropping
negative speed for shape 40808, dropping
speed above 80 for shape 40844, dropping
negative speed for shape 40844, dropping
speed above 80 for shape 40853, dropping


TopologyException: assigned depths do not match at -121992.84082232323 62752.425028798818
/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:515: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [34]:
# m

In [17]:
sac_rt.endpoint_delay_view = (sac_rt.stop_delay_view
                              >> group_by(_.trip_id)
                              >> filter(_.stop_sequence == _.stop_sequence.max())
                              >> ungroup()
                              >> mutate(arrival_hour = _.arrival_time.apply(lambda x: x.hour))
                              # >> add_count(_.direction_id, _.route_id, _.arrival_hour)
                             )

In [18]:
endpoint_delays = (sac_rt.endpoint_delay_view
                   >> group_by(_.direction_id, _.route_id, _.arrival_hour)
                   >> summarize(n_trips = _.route_id.size, mean_end_delay = _.delay.mean())
                   # >> ungroup()
                   # >> mutate(mean_end_delay = _.delay.mean())
                  )

In [19]:
endpoint_delays >> filter(_.n_trips > 2) >> arrange(-_.mean_end_delay)

,direction_id,route_id,arrival_hour,n_trips,mean_end_delay
335,0,081,16,4,0 days 00:07:24.259017
847,1,081,15,5,0 days 00:06:19.786659400
135,0,026,14,3,0 days 00:05:55.245385
331,0,081,12,4,0 days 00:05:48.326088750
333,0,081,14,4,0 days 00:05:19.730590500
...,...,...,...,...,...
757,1,062,19,3,0 days 00:00:00
762,1,067,9,3,0 days 00:00:00
766,1,067,13,3,0 days 00:00:00
838,1,081,6,3,0 days 00:00:00


In [20]:
sac_rt.endpoint_delay_view >> filter(_.direction_id == '1', _.route_id == '081', _.arrival_hour == 15)

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,direction_id,actual_time,delay,arrival_hour
69,4810,FLORIN RD & WINDWARD WAY (NB),POINT (-134220.994 55695.322),40806,18069.270655,8822785355802769737,992880,35,2022-02-08 15:34:00,081,1,2022-02-08 15:41:00.874288,0 days 00:07:00.874288,15
98,4810,FLORIN RD & WINDWARD WAY (NB),POINT (-134220.994 55695.322),40806,18069.270655,-4507200914102975562,992881,35,2022-02-08 15:00:00,081,1,2022-02-08 15:00:59.084276,0 days 00:00:59.084276,15
100,1611,65TH ST & 53RD AVE (SB),POINT (-124415.884 54987.452),40803,6307.003279,-1490517597643705058,992839,11,2022-02-08 15:42:00,081,1,2022-02-08 15:50:43.678278,0 days 00:08:43.678278,15
1649,4810,FLORIN RD & WINDWARD WAY (NB),POINT (-134220.994 55695.322),40805,11207.174508,7873962454332529141,992849,24,2022-02-08 15:46:00,081,1,2022-02-08 16:00:55.296455,0 days 00:14:55.296455,15
1652,4810,FLORIN RD & WINDWARD WAY (NB),POINT (-134220.994 55695.322),40805,11207.174508,-3789738650081171942,992850,24,2022-02-08 15:15:00,081,1,2022-02-08 15:13:35.381493,0 days 00:00:00,15


In [21]:
florin1 = sac_rt.position_interpolators['992810']['rt']
# florin1.detailed_speed_map()

In [35]:
florin2 = sac_rt.position_interpolators['992849']['rt']
florin2.detailed_speed_map()

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:105: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [23]:
sac_rt.rt_trips = sac_rt.rt_trips >> inner_join(_, sac_rt.endpoint_delay_view >> select(_.trip_id, _.endpoint_delay == _.delay,
                                                                      _.arrival_hour)
                              , on = 'trip_id')

In [24]:
sac_rt.rt_trips >> arrange(-_.endpoint_delay)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,median_time,direction,mean_speed_mph,endpoint_delay,arrival_hour
1217,273,0,2022-02-08,7604500401212340186,991485,051,0,40770,2021-12-17,2099-01-01,06:33:00,Northbound,11.435978,0 days 00:52:07.711814,6
1543,273,0,2022-02-08,-6338363960220385199,993897,138,0,40847,2021-12-17,2099-01-01,07:04:30,Westbound,24.915089,0 days 00:40:00.024699,6
553,273,0,2022-02-08,-2947005468291523101,990964,026,0,40760,2021-12-17,2099-01-01,17:08:00,Southbound,9.578548,0 days 00:23:58.102561,17
590,273,0,2022-02-08,2613461417951518281,993889,138,0,40847,2021-12-17,2099-01-01,09:56:30,Westbound,58.745901,0 days 00:23:16.722503,9
823,273,0,2022-02-08,-8356190538517605376,993905,138,1,40853,2021-12-17,2099-01-01,07:47:30,Eastbound,44.161993,0 days 00:22:26.521702,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,273,0,2022-02-08,4434607073140252276,992219,067,1,40786,2021-12-17,2099-01-01,11:19:00,Southbound,15.688883,0 days 00:00:00,11
1927,273,0,2022-02-08,2395426288088610260,989934,011,1,40739,2021-12-17,2099-01-01,15:47:00,Northbound,13.537608,0 days 00:00:00,16
1928,273,0,2022-02-08,3206225287228501872,989935,011,1,40739,2021-12-17,2099-01-01,15:23:00,Northbound,13.754458,0 days 00:00:00,15
1929,273,0,2022-02-08,2754952228455939726,990597,021,1,40752,2021-12-17,2099-01-01,20:57:30,Northbound,19.954131,0 days 00:00:00,21


In [25]:
## awesome example here
fwy_slow = sac_rt.position_interpolators['990964']['rt']
# fwy_slow.detailed_speed_map()

In [26]:
fwy_slow.detailed_speed_map()

/home/jovyan/data-analyses/rt_delay/rt_analysis.py:105: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

